In [1317]:
##Enter the number of new orders up to 100 here
number_of_new_orders = 20

In [1318]:
import requests
import os
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gspread
import df2gspread as d2g
from google.oauth2.service_account import Credentials
from gspread_dataframe import set_with_dataframe



#gets list of orders from shipstation
url = "https://ssapi.shipstation.com/orders?sortDir=DESC"


payload = {}

headers = {
  'Host': 'ssapi.shipstation.com',
  'Authorization': 'XXXXXXXXXXXXXXXXXXXXX' #your api_key and api_secret UTF-8 encoded
}

response = requests.get(url, headers=headers, data = payload)


#gets list of stores from shipstation
url = "https://ssapi.shipstation.com/stores"

payload = {}
headers = {
  'Host': 'ssapi.shipstation.com',
  'Authorization': 'XXXXXXXXXXXXXXXXXXXXX' #your api_key and api_secret UTF-8 encoded
}

stores = requests.request("GET", url, headers=headers, data = payload)
response
stores


<Response [200]>

In [1319]:
stores = stores.json()

In [1320]:
stores_df = pd.DataFrame(stores)

In [1321]:
stores_df = stores_df[['storeId', 'storeName']]

In [1322]:
json = response.json()
json = json['orders']

In [1323]:
df = pd.DataFrame(json)

In [1324]:
df = df.reindex(index=df.index[::-1])

In [1325]:
df = df[['orderDate', 'billTo', 'amountPaid', 'advancedOptions']]

In [1326]:
def slice_date (col):
    '''
    This function is to slice the orderDate column.
    '''
    date = col[5:10]
    
    return date
    

In [1327]:
df['orderDate'] = df['orderDate'].apply(slice_date)

In [1328]:
def date_to_sheet_format(date_to_convert):
    '''
    This function takes a dictionary of <Month Abreviation:Month Number>, and
    generates the Month_abreviation when a speicific label is given
    Arguments: 
        dict: a dictionary <key: value>
            with month abreviations as the key and numerical labels as the value
        val: a string to be converted to a date
    Return: 
        A string that is in the format of '10-Mar'
    '''
    months_dict = {
     'Jan': 1,
     'Feb': 2,
     'Mar': 3,
     'Apr': 4,
     'May': 5,
     'Jun': 6,
     'Jul': 7,
     'Aug': 8,
     'Sep': 9,
     'Oct': 10,
     'Nov': 11,
     'Dec': 12
    }
    key_list = list(months_dict.keys())
    val_list = list(months_dict.values())
    day = int(date_to_convert[3:])
    month_number = int(date_to_convert[:2])  
    month_abrev = key_list[val_list.index(month_number)]
    
    return str(day)+ '-' + month_abrev

In [1329]:
df['orderDate'] = df['orderDate'].apply(date_to_sheet_format)

In [1330]:
def name_to_sheet_format (billTo_to_convert):
    '''
    This function takes a dictionary of billTo info from shipstation and
    returns a string in the format of 'lastName, firstName' to be inserted 
    into the spreadsheet.
    Arguments: 
        billto_to_convert: col of all billTo info from shipstation
    Return: 
        A string in the format of 'lastName, firstName'
    '''
    return list(billTo_to_convert.values())[0].split(' ', 1)[1].capitalize()+', '+ list(billTo_to_convert.values())[0].split(' ', 1)[0].capitalize()
    
    

In [1331]:
df['billTo'] = df['billTo'].apply(name_to_sheet_format)

In [1332]:
def ammountPaid_to_sheet_format (ammountPaid_to_convert):
    '''
    This function takes the ammountPaid float from shipstation and
    returns a string in the format of '$ammountPaid' to be inserted 
    into the spreadsheet.
    Arguments: 
        ammountPaid_to_convert: col of ammountPaid values
    Return: 
        A string in the format of '$ammountPaid'
    '''
    return '$'+str(ammountPaid_to_convert)

In [1333]:
df['amountPaid'] = df['amountPaid'].apply(ammountPaid_to_sheet_format)

In [1334]:
def get_storeId_from_advancedOptions (advancedOptions_to_convert):
    '''
    This function takes the advancedOptions dictionary from shipstation and
    returns a unique storeId int.
    Arguments: 
        advancedOptions_to_convert: col of advancedOption dictionaries
    Return: 
        A unique storeId int
    '''
    
    return list(advancedOptions_to_convert.values())[7]

In [1335]:
df['advancedOptions'] = df['advancedOptions'].apply(get_storeId_from_advancedOptions)

In [1336]:
df = df.rename(columns={"advancedOptions": "storeId"})

In [1337]:
#This was the only place that I was able to find website name in the shipstaition order data model
def website_to_sheet_format (website_to_convert):
    '''
    This function takes a dictionary of billTo info from shipstation and
    returns a string in the format of 'lastName, firstName' to be inserted 
    into the spreadsheet.
    Arguments: 
        billto_to_convert: col of all billTo info from shipstation
    Return: 
        A string in the format of 'lastName, firstName'
    '''
    site_dict = list(stores_df.set_index('storeId').to_dict().values())[0]
    site_abbreviation_list = ['Whirlpool Tub Parts', 'Whirlpooltubparts.net', 'Hottubparts.net', 'iPoolandSpaParts.net', 'HydraBathParts.com                           ', 'Whirlpoolpumps.com', 'LascoTubParts.com', 'PDCSpaParts.com                                   ', 'iHottubparts.net', 'Whirlpooltubparts.biz', 'iHotTubParts.com']
    site_abbreviation_dict = {
     'WPTP': 'Whirlpool Tub Parts',
     'WPTP.NET': 'Whirlpooltubparts.net',
     'HTP': 'Hottubparts.net',
     'Jacuzzi': 'iPoolandSpaParts.net',
     'HydraBath': 'HydraBathParts.com                           ',
     'PUMPS': 'Whirlpoolpumps.com',
     'Lasco': 'LascoTubParts.com',
     'PDC': 'PDCSpaParts.com                                   ',
     'Sundance': 'iHottubparts.net',
     'WPTP.BIZ': 'Whirlpooltubparts.biz',
     'HydroQuip': 'iHotTubParts.com'
    }
    key_list = list(site_dict.keys())
    val_list = list(site_dict.values()) 
    site = val_list[key_list.index(website_to_convert)]
    
    if site in site_abbreviation_list:
        #change to abbreviation based on site
        key_list = list(site_abbreviation_dict.keys())
        val_list = list(site_abbreviation_dict.values()) 
        return key_list[val_list.index(site)]
    else:
        #keep site url as name
        return site
    


In [1338]:
df['storeId'] = df['storeId'].apply(website_to_sheet_format)

In [1339]:
final_df = df[['orderDate', 'billTo', 'storeId', 'amountPaid']]

In [1340]:
scopes = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']

credentials = Credentials.from_service_account_file('your_service_account_json_file', scopes=scopes)

gc = gspread.authorize(credentials)

#gauth = GoogleAuth()
#drive = GoogleDrive(gauth)

# open a google sheet
gs = gc.open_by_key('your_google_sheet_key')
# select a work sheet from its name
worksheet1 = gs.worksheet('Sheet1')

In [1341]:
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=final_df.tail(number_of_orders), include_index=False,
include_column_header=True, resize=True)